In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import v2
from torchmetrics.classification import MulticlassAccuracy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
no_epochs = 50
learning_rate = 0.001
batch_size = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

acc_function = MulticlassAccuracy(num_classes=102, average='micro').to(device)
loss_fn = nn.CrossEntropyLoss().to(device)

SEED = 42
np.random.seed(SEED)
gen = torch.manual_seed(SEED)

cuda:0


In [3]:
# Data Augmentation
train_transforms = v2.Compose([
    v2.RandomRotation(30),
    v2.RandomResizedCrop(224),
    v2.RandomHorizontalFlip(),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

default_transforms = v2.Compose([
    models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
])

flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=train_transforms)
flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=default_transforms)
flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=default_transforms)


c:\Builds\CZ4042-group-project\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [4]:
def get_data_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(flowers_train, batch_size=batch_size, shuffle=True, generator=gen)
    test_loader = torch.utils.data.DataLoader(flowers_test, batch_size=batch_size, shuffle=True, generator=gen)
    val_loader = torch.utils.data.DataLoader(flowers_val, batch_size=batch_size, shuffle=True, generator=gen)
    return train_loader, test_loader, val_loader

In [5]:
# Early stopping based on accuracy
class AccuracyEarlyStopper:
    def __init__(self, patience=3, min_delta=0.5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_accuracy = 0

    def early_stop(self, validation_accuracy):
        if validation_accuracy > (self.max_validation_accuracy + self.min_delta):
            self.max_validation_accuracy = validation_accuracy
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [6]:
def train(model, optimizer, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        running_loss_value += loss.item()
        loss.backward()
        optimizer.step()
    return running_loss_value / len(dataloader)

def test_eval(model, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    running_acc_value = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            acc = acc_function(outputs, labels)
            running_loss_value += loss.item()
            running_acc_value += acc.item()
    running_acc_value /= len(dataloader)
    running_loss_value /= len(dataloader)
    return running_acc_value*100, running_loss_value

def train_eval_test(model, train_dataloader, val_dataloader, test_dataloader, no_epochs=10):
    es = AccuracyEarlyStopper()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr, training_time = [], [], [], [], []
    for i in range(no_epochs):
        start = datetime.datetime.now()
        train_loss = train(model, optimizer, train_dataloader)
        end = datetime.datetime.now()
        time_taken = (end - start).total_seconds()
        eval_acc, eval_loss = test_eval(model, val_dataloader)
        print(f'Epoch {i+1} Train Loss: {train_loss:>8f}, Eval Accuracy: {eval_acc:>0.2f}%, Eval Loss: {eval_loss:>8f}, Time Taken: {time_taken:>0.2f}s')
        train_loss_arr.append(train_loss)
        eval_loss_arr.append(eval_loss)
        eval_acc_arr.append(eval_acc)
        training_time.append(time_taken)
        if es.early_stop(eval_acc):
            print('Early stopping activated')
            break
        if i == 1:
            break
    test_acc, test_loss = test_eval(model, test_dataloader)
    print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")
    return train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr, test_acc, test_loss, training_time

In [7]:
def create_VGG_model():
    model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
    new_classifier_head = nn.Sequential(
        nn.Linear(25088, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 102)
    )
    
    # for param in model.parameters():
    #     param.requires_grad = False
        
    model.classifier = new_classifier_head

    return model.to(device=device)

def create_resnet_model(size=50):
    match size:
        case 18:
            model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            new_fc = nn.Linear(512,102)
        case 34:
            model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
            new_fc = nn.Linear(512,102)
        case 50:
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            new_fc = nn.Linear(2048, 102)
        case 101:
            model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
            new_fc = nn.Linear(2048,102)
        case _:
            print("Invalid Size, defaulting to 50")
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            new_fc = nn.Linear(2048, 102)
    
    # for param in model.parameters():
    #     param.requires_grad = False
        
    model.fc = new_fc
    
    return model.to(device)

def create_efficientnet_model(size):
    match size:
        case 's':
            model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
        case 'm':
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        case 'l':
            model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.DEFAULT)
        case _:
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)



    # for param in model.parameters():
    #     param.requires_grad = False
        
    new_classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(1280, 102)
    )
    
    model.classifier = new_classifier
    
    return model.to(device)

In [8]:
train_data_loader, test_data_loader, val_data_loader = get_data_loader(batch_size)

In [9]:
VGG_model = create_VGG_model()
print("VGG16")
VGG_train_acc, VGG_train_loss, VGG_eval_acc, VGG_eval_loss, VGG_test_acc, VGG_test_loss, VGG_training_time = train_eval_test(
    VGG_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del VGG_model

VGG16


In [ ]:
ResNet_18_model = create_resnet_model(18)
print("RESNET18")
ResNet18_train_acc, ResNet18_train_loss, ResNet18_eval_acc, ResNet18_eval_loss, ResNet18_test_acc, ResNet18_test_loss, ResNet18_training_time = train_eval_test(
    ResNet_18_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del ResNet_18_model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Elliot/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


RESNET
Epoch 1 Train Loss: 4.756111, Eval Accuracy: 5.48%, Eval Loss: 4.367081
Epoch 2 Train Loss: 4.244765, Eval Accuracy: 19.80%, Eval Loss: 3.967921
Epoch 3 Train Loss: 3.828414, Eval Accuracy: 37.64%, Eval Loss: 3.559131
Epoch 4 Train Loss: 3.423972, Eval Accuracy: 50.29%, Eval Loss: 3.200543
Epoch 5 Train Loss: 3.071233, Eval Accuracy: 62.56%, Eval Loss: 2.874337
Epoch 6 Train Loss: 2.756047, Eval Accuracy: 65.19%, Eval Loss: 2.592432
Epoch 7 Train Loss: 2.460320, Eval Accuracy: 70.11%, Eval Loss: 2.345962
Epoch 8 Train Loss: 2.228341, Eval Accuracy: 72.85%, Eval Loss: 2.133216
Epoch 9 Train Loss: 2.022457, Eval Accuracy: 74.89%, Eval Loss: 1.950958
Epoch 10 Train Loss: 1.834905, Eval Accuracy: 77.56%, Eval Loss: 1.787383
Epoch 11 Train Loss: 1.680075, Eval Accuracy: 79.12%, Eval Loss: 1.657058
Epoch 12 Train Loss: 1.570435, Eval Accuracy: 79.90%, Eval Loss: 1.551310
Epoch 13 Train Loss: 1.437939, Eval Accuracy: 79.71%, Eval Loss: 1.448478
Epoch 14 Train Loss: 1.379880, Eval Accur

In [ ]:
ResNet_34_model = create_resnet_model(34)
print("RESNET34")
ResNet34_train_acc, ResNet34_train_loss, ResNet34_eval_acc, ResNet34_eval_loss, ResNet34_test_acc, ResNet34_test_loss, ResNet34_training_time = train_eval_test(
    ResNet_34_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del ResNet_34_model

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\Elliot/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100.0%


RESNET
Epoch 1 Train Loss: 4.781655, Eval Accuracy: 7.05%, Eval Loss: 4.343193
Epoch 2 Train Loss: 4.230287, Eval Accuracy: 18.33%, Eval Loss: 3.936898
Epoch 3 Train Loss: 3.761862, Eval Accuracy: 37.76%, Eval Loss: 3.514868
Epoch 4 Train Loss: 3.368629, Eval Accuracy: 49.81%, Eval Loss: 3.146169
Epoch 5 Train Loss: 3.077458, Eval Accuracy: 61.66%, Eval Loss: 2.815254
Epoch 6 Train Loss: 2.717633, Eval Accuracy: 66.46%, Eval Loss: 2.534502
Epoch 7 Train Loss: 2.405202, Eval Accuracy: 70.28%, Eval Loss: 2.278708
Epoch 8 Train Loss: 2.178649, Eval Accuracy: 72.36%, Eval Loss: 2.063467
Epoch 9 Train Loss: 1.972587, Eval Accuracy: 74.90%, Eval Loss: 1.880205
Epoch 10 Train Loss: 1.793364, Eval Accuracy: 77.15%, Eval Loss: 1.727702
Epoch 11 Train Loss: 1.655780, Eval Accuracy: 78.24%, Eval Loss: 1.601576
Epoch 12 Train Loss: 1.496205, Eval Accuracy: 80.09%, Eval Loss: 1.481993
Epoch 13 Train Loss: 1.376546, Eval Accuracy: 81.15%, Eval Loss: 1.382909
Epoch 14 Train Loss: 1.314042, Eval Accur

In [ ]:
ResNet_50_model = create_resnet_model(50)
print("RESNET50")
ResNet50_train_acc, ResNet50_train_loss, ResNet50_eval_acc, ResNet50_eval_loss, ResNet50_test_acc, ResNet50_test_loss, ResNet50_training_time = train_eval_test(
    ResNet_50_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del ResNet_50_model

RESNET
Epoch 1 Train Loss: 4.559777, Eval Accuracy: 17.35%, Eval Loss: 4.293632
Epoch 2 Train Loss: 4.145323, Eval Accuracy: 48.33%, Eval Loss: 3.954787
Epoch 3 Train Loss: 3.792295, Eval Accuracy: 66.16%, Eval Loss: 3.654354
Epoch 4 Train Loss: 3.475689, Eval Accuracy: 73.27%, Eval Loss: 3.368963
Epoch 5 Train Loss: 3.190637, Eval Accuracy: 74.84%, Eval Loss: 3.093798
Epoch 6 Train Loss: 2.930863, Eval Accuracy: 77.07%, Eval Loss: 2.880001
Epoch 7 Train Loss: 2.691964, Eval Accuracy: 78.94%, Eval Loss: 2.667114
Epoch 8 Train Loss: 2.451674, Eval Accuracy: 80.39%, Eval Loss: 2.450377
Epoch 9 Train Loss: 2.237405, Eval Accuracy: 81.09%, Eval Loss: 2.292343
Epoch 10 Train Loss: 2.121731, Eval Accuracy: 81.88%, Eval Loss: 2.131230
Epoch 11 Train Loss: 1.932281, Eval Accuracy: 83.04%, Eval Loss: 1.996986
Epoch 12 Train Loss: 1.754039, Eval Accuracy: 82.55%, Eval Loss: 1.879992
Epoch 13 Train Loss: 1.628438, Eval Accuracy: 82.83%, Eval Loss: 1.772269
Epoch 14 Train Loss: 1.529859, Eval Accu

In [ ]:
ResNet_101_model = create_resnet_model(101)
print("RESNET101")
ResNet101_train_acc, ResNet101_train_loss, ResNet101_eval_acc, ResNet101_eval_loss, ResNet101_test_acc, ResNet101_test_loss, ResNet101_training_time = train_eval_test(
    ResNet_101_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del ResNet_101_model

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to C:\Users\Elliot/.cache\torch\hub\checkpoints\resnet101-cd907fc2.pth
100.0%


RESNET
Epoch 1 Train Loss: 4.566488, Eval Accuracy: 16.48%, Eval Loss: 4.235667
Epoch 2 Train Loss: 4.037764, Eval Accuracy: 42.57%, Eval Loss: 3.827048
Epoch 3 Train Loss: 3.618826, Eval Accuracy: 60.98%, Eval Loss: 3.446374
Epoch 4 Train Loss: 3.229564, Eval Accuracy: 68.04%, Eval Loss: 3.109408
Epoch 5 Train Loss: 2.898887, Eval Accuracy: 71.36%, Eval Loss: 2.801347
Epoch 6 Train Loss: 2.593585, Eval Accuracy: 74.33%, Eval Loss: 2.543025
Epoch 7 Train Loss: 2.310902, Eval Accuracy: 76.85%, Eval Loss: 2.312585
Epoch 8 Train Loss: 2.047343, Eval Accuracy: 78.74%, Eval Loss: 2.119800
Epoch 9 Train Loss: 1.853017, Eval Accuracy: 78.74%, Eval Loss: 1.953272
Epoch 10 Train Loss: 1.715472, Eval Accuracy: 78.81%, Eval Loss: 1.820706
Epoch 11 Train Loss: 1.499737, Eval Accuracy: 79.90%, Eval Loss: 1.688021
Epoch 12 Train Loss: 1.389339, Eval Accuracy: 80.68%, Eval Loss: 1.589905
Epoch 13 Train Loss: 1.302078, Eval Accuracy: 80.59%, Eval Loss: 1.503033
Epoch 14 Train Loss: 1.181725, Eval Accu

In [ ]:
efficientnet_s_model = create_efficientnet_model('s')

print("EFFICIENTNET_S")

ENs_train_acc, ENs_train_loss, ENs_eval_acc, ENs_eval_loss, ENs_test_acc, ENs_test_loss, ENs_training_time = train_eval_test(
    efficientnet_s_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del efficientnet_s_model

EFFICIENTNET
Epoch 1 Train Loss: 4.574721, Eval Accuracy: 12.42%, Eval Loss: 4.353661
Epoch 2 Train Loss: 4.185564, Eval Accuracy: 33.15%, Eval Loss: 4.031021
Epoch 3 Train Loss: 3.860585, Eval Accuracy: 46.29%, Eval Loss: 3.739001
Epoch 4 Train Loss: 3.542766, Eval Accuracy: 53.71%, Eval Loss: 3.490047
Epoch 5 Train Loss: 3.278103, Eval Accuracy: 59.00%, Eval Loss: 3.235458
Epoch 6 Train Loss: 3.031440, Eval Accuracy: 62.44%, Eval Loss: 3.006856
Epoch 7 Train Loss: 2.782584, Eval Accuracy: 66.27%, Eval Loss: 2.804595
Epoch 8 Train Loss: 2.568160, Eval Accuracy: 68.70%, Eval Loss: 2.628160
Epoch 9 Train Loss: 2.379220, Eval Accuracy: 68.03%, Eval Loss: 2.468820
Epoch 10 Train Loss: 2.258689, Eval Accuracy: 69.42%, Eval Loss: 2.334562
Epoch 11 Train Loss: 2.072847, Eval Accuracy: 71.96%, Eval Loss: 2.208270
Epoch 12 Train Loss: 1.962521, Eval Accuracy: 72.17%, Eval Loss: 2.104919
Epoch 13 Train Loss: 1.806893, Eval Accuracy: 72.62%, Eval Loss: 2.006330
Epoch 14 Train Loss: 1.714660, Eva

In [ ]:
efficientnet_m_model = create_efficientnet_model('m')

print("EFFICIENTNET_M")

ENm_train_acc, ENm_train_loss, ENm_eval_acc, ENm_eval_loss, ENm_test_acc, ENm_test_loss, ENm_training_time = train_eval_test(
    efficientnet_m_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del efficientnet_m_model

EFFICIENTNET
Epoch 1 Train Loss: 4.611644, Eval Accuracy: 7.65%, Eval Loss: 4.419560
Epoch 2 Train Loss: 4.293189, Eval Accuracy: 23.92%, Eval Loss: 4.170458
Epoch 3 Train Loss: 4.053287, Eval Accuracy: 31.60%, Eval Loss: 3.978391
Epoch 4 Train Loss: 3.802157, Eval Accuracy: 44.62%, Eval Loss: 3.749610
Epoch 5 Train Loss: 3.574698, Eval Accuracy: 48.72%, Eval Loss: 3.571569
Epoch 6 Train Loss: 3.369551, Eval Accuracy: 47.59%, Eval Loss: 3.400626
Epoch 7 Train Loss: 3.186699, Eval Accuracy: 49.02%, Eval Loss: 3.251072
Epoch 8 Train Loss: 2.999997, Eval Accuracy: 54.34%, Eval Loss: 3.101571
Epoch 9 Train Loss: 2.876097, Eval Accuracy: 55.20%, Eval Loss: 2.959369
Epoch 10 Train Loss: 2.668106, Eval Accuracy: 55.37%, Eval Loss: 2.853476
Epoch 11 Train Loss: 2.588547, Eval Accuracy: 58.12%, Eval Loss: 2.717815
Epoch 12 Train Loss: 2.405632, Eval Accuracy: 57.84%, Eval Loss: 2.642203
Epoch 13 Train Loss: 2.324905, Eval Accuracy: 59.61%, Eval Loss: 2.546059
Epoch 14 Train Loss: 2.254833, Eval

In [ ]:
efficientnet_l_model = create_efficientnet_model('l')

print("EFFICIENTNET_L")

ENl_train_acc, ENl_train_loss, ENl_eval_acc, ENl_eval_loss, ENl_test_acc, ENl_test_loss, ENl_training_time = train_eval_test(
    efficientnet_l_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

del efficientnet_l_model

EFFICIENTNET
Epoch 1 Train Loss: 4.590027, Eval Accuracy: 15.59%, Eval Loss: 4.096957
Epoch 2 Train Loss: 3.842896, Eval Accuracy: 39.08%, Eval Loss: 3.421711
Epoch 3 Train Loss: 3.141942, Eval Accuracy: 63.46%, Eval Loss: 2.797723
Epoch 4 Train Loss: 2.599770, Eval Accuracy: 70.60%, Eval Loss: 2.338389
Epoch 5 Train Loss: 2.141581, Eval Accuracy: 76.46%, Eval Loss: 1.960250
Epoch 6 Train Loss: 1.845971, Eval Accuracy: 79.10%, Eval Loss: 1.681222
Epoch 7 Train Loss: 1.584890, Eval Accuracy: 83.33%, Eval Loss: 1.475755
Epoch 8 Train Loss: 1.366856, Eval Accuracy: 83.62%, Eval Loss: 1.310721
Epoch 9 Train Loss: 1.229933, Eval Accuracy: 84.31%, Eval Loss: 1.166734
Epoch 10 Train Loss: 1.117605, Eval Accuracy: 85.98%, Eval Loss: 1.079130
Epoch 11 Train Loss: 0.930863, Eval Accuracy: 86.49%, Eval Loss: 0.985865
Epoch 12 Train Loss: 0.943139, Eval Accuracy: 87.25%, Eval Loss: 0.923959
Epoch 13 Train Loss: 0.872363, Eval Accuracy: 89.13%, Eval Loss: 0.862135
Epoch 14 Train Loss: 0.771146, Eva

In [ ]:
eval_acc = [VGG_eval_acc, ResNet18_eval_acc, ResNet34_eval_acc, ResNet50_eval_acc, ResNet101_eval_acc, ENs_eval_acc, ENm_eval_acc, ENl_eval_acc]
eval_acc = [np.mean(x) for x in eval_acc]
test_acc = [VGG_test_acc, ResNet18_test_acc, ResNet34_test_acc, ResNet50_test_acc, ResNet101_test_acc, ENs_test_acc, ENm_test_acc, ENl_test_acc]
time_taken = [VGG_training_time, ResNet18_training_time, ResNet34_training_time, ResNet50_training_time, ResNet101_training_time, ENs_training_time, ENm_training_time, ENl_training_time]
time_taken = [np.mean(x) for x in time_taken]
names = ['VGG16', 'ResNet18', 'ResNet34', 'ResNet50', 'ResNet101', 'EfficientNet_S', 'EfficientNet_M', 'EfficientNet_L']
colours = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray']

In [ ]:
plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.title('Validation Accuracies')
plt.scatter(names, eval_acc, color=colours)
plt.subplot(132)
plt.title('Test Accuracies')
plt.scatter(names, test_acc, color=colours)
plt.subplot(133)
plt.title('Training Times')
plt.scatter(names, time_taken, color=colours)
plt.show()